# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khunti,23.0833,85.2833,23.06,98,100,7.27,IN,1722608144
1,1,puerto natales,-51.7236,-72.4875,0.25,100,100,2.57,CL,1722608046
2,2,chonchi,-42.6128,-73.8092,8.56,90,99,3.13,CL,1722608146
3,3,atafu village,-8.5421,-172.5159,28.12,74,70,6.45,TK,1722608148
4,4,nyrsko,49.2939,13.1435,19.81,92,100,4.17,CZ,1722608149


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27.0) & (city_data_df["Max Temp"] >= 21.0)& (city_data_df["Wind Speed"] <= 4.5)
                                 & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_condition_df = ideal_condition_df.dropna()

# Display sample data
ideal_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,petauke,-14.2426,31.3253,26.72,19,0,3.92,ZM,1722608253
108,108,floro,61.5996,5.0328,24.04,68,0,4.12,NO,1722608271
128,128,duluth,46.7833,-92.1066,24.63,78,0,3.60,US,1722608179
141,141,cottage grove,44.8277,-92.9438,25.07,79,0,2.06,US,1722608310
160,160,bandrele,-12.9067,45.1914,26.86,61,0,4.12,YT,1722608334
177,177,coeur d'alene,47.6777,-116.7805,21.17,67,0,4.12,US,1722608253
203,203,chibougamau,49.9168,-74.3659,24.79,65,0,3.09,CA,1722608238
235,235,saint-pierre,-21.3393,55.4781,22.82,60,0,3.09,RE,1722608422
240,240,malpica,43.3228,-8.8105,22.16,68,0,2.91,ES,1722608428
260,260,khanty-mansiysk,61.0042,69.0019,24.06,60,0,2.00,RU,1722608292


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
92,petauke,ZM,-14.2426,31.3253,19,
108,floro,NO,61.5996,5.0328,68,
128,duluth,US,46.7833,-92.1066,78,
141,cottage grove,US,44.8277,-92.9438,79,
160,bandrele,YT,-12.9067,45.1914,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
      "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
petauke - nearest hotel: Chimwemwe Lodge
floro - nearest hotel: Comfort Hotel Victoria Florø
duluth - nearest hotel: Radisson Hotel Duluth-Harborview
cottage grove - nearest hotel: Country Inn & Suites
bandrele - nearest hotel: Les Baobabs
coeur d'alene - nearest hotel: Roosevelt Inn
chibougamau - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
malpica - nearest hotel: Pensión Panchito
khanty-mansiysk - nearest hotel: Гамма
tonganoxie - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
ramos arizpe - nearest hotel: Quinta Royal
lebowakgomo - nearest hotel: Chuene Resort
lockwood - nearest hotel: Holiday Inn Express
sinop - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
tsiombe - nearest hotel: No hotel found
arafat - nearest hotel: No hotel found
stewartville - nearest hotel: AmericInn by Wyndham Rochester Airport
north platte - nearest hotel: Howard Johnson Inn - North Platte
chipinge - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
92,petauke,ZM,-14.2426,31.3253,19,Chimwemwe Lodge
108,floro,NO,61.5996,5.0328,68,Comfort Hotel Victoria Florø
128,duluth,US,46.7833,-92.1066,78,Radisson Hotel Duluth-Harborview
141,cottage grove,US,44.8277,-92.9438,79,Country Inn & Suites
160,bandrele,YT,-12.9067,45.1914,61,Les Baobabs
177,coeur d'alene,US,47.6777,-116.7805,67,Roosevelt Inn
203,chibougamau,CA,49.9168,-74.3659,65,No hotel found
235,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
240,malpica,ES,43.3228,-8.8105,68,Pensión Panchito
260,khanty-mansiysk,RU,61.0042,69.0019,60,Гамма


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    alpha = 0.5,
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [ ]:
#You can find above what are the best hotels to go on vacation as per the criteria defined in the challenge